<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" property="dct:title"><b>Modeling with Gurobi: the basics</b></span> by <a xmlns:cc="http://creativecommons.org/ns#" href="http://mate.unipv.it/gualandi" property="cc:attributionName" rel="cc:attributionURL">Stefano Gualandi</a> is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Creative Commons Attribution 4.0 International License</a>.<br />Based on a work at <a xmlns:dct="http://purl.org/dc/terms/" href="https://github.com/mathcoding/opt4ds" rel="dct:source">https://github.com/mathcoding/opt4ds</a>.

**NOTE:** Execute the following command whenever running this script on a Google Colab.

In [3]:
# Run if on Colab
# %pip install gurobipy

In [4]:
import gurobipy as grb
import matplotlib.pyplot as plt

# 2. Writing LP models with Gurobi
In this notebook, we show how to use Gurobi to write general LPs problems.

## 2.1 Steel Production Planning

In this notebook, we explain how to solve the **Linear Programming** problem that we have written to solve the Steel Planning problem during the class (see the slides on KIRO). This problem is given as Exercise 1.1 in Chapter 1 of [Linear Programming, Foundations and Extensions](https://link.springer.com/book/10.1007/978-1-4614-7630-6) by [R.J. Vanderbei](https://vanderbei.princeton.edu/).

We show below how to use [Gurobi](https://www.gurobi.com/academia/academic-program-and-licenses/) to define the **variables**, the **objective function**, and the **constraints**.

**CHECK THE NOTEBOOK FOR THIS EXERCISE TO GET THE BASICS OF THE GUROBI SYNTAX**

## 2.2 Lego Planning Problem
As a first exercise, you have to solve the **Linear Programming (LP)** problem that we have written to model the Lego Planning problem (see the slides on KIRO):

$$
\begin{align}
\max \quad & 8 c + 11 t  \\
 \quad & 2c + 2t \leq 24 \\
& c + 2 t \leq 18\\
& c \geq 0\\
& t \geq 0
\end{align}
$$

You have to use Gurobi to define the **variables**, the **objective function**, and the **constraints**.

**EXERCISE 1:** Using the template for the *Steel Production Planning Problem* solve the LEGO instance defined above.

In [5]:
# Write your own script
from gurobipy import Model, GRB
model = Model()

p = [8,11] # price vector
c = model.addVar(lb = 0, obj = p[0], vtype = GRB.INTEGER, name = "c")
t = model.addVar(lb = 0, obj = p[1], vtype = GRB.INTEGER, name = "t")


model.ModelSense = GRB.MAXIMIZE


A = [[2,2],[1,2]]
b = [24,18]

# this is good...
# model.addConstr(2*c+2*t <= 24)
# model.addConstr(c+2*t <= 18)

# this is better

for Ai, bi in zip(A,b): # zip(A,b) returns list made by [(a1, b1), (a2, b2)...]
  model.addConstr(Ai[0]*c+Ai[1]*t <= bi)

model.update()
model.write("lego.lp")

Set parameter Username
Set parameter LicenseID to value 2587342
Academic license - for non-commercial use only - expires 2025-11-19


In [6]:
model.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i3-1005G1 CPU @ 1.20GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 2 rows, 2 columns and 4 nonzeros
Model fingerprint: 0x11438d89
Variable types: 0 continuous, 2 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [8e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 2e+01]
Found heuristic solution: objective 96.0000000
Presolve time: 0.01s
Presolved: 2 rows, 2 columns, 4 nonzeros
Variable types: 0 continuous, 2 integer (0 binary)

Root relaxation: objective 1.140000e+02, 2 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     114.0000000  114.00

In [7]:
print("status: ", model.Status) # 2 is good
print("objective: ", model.ObjVal)
print("Decision variables:")
print("\t c:", c.X)
print("\t t:", t.X)

status:  2
objective:  114.0
Decision variables:
	 c: 6.0
	 t: 6.0


it's the solution we were looking for!!!

**EXERCISE 2:** Modify the previous script to solve the second version of the Lego Planning problem:

$$
\begin{align}
\max \quad & 8 c + 11 t + 15 s \\
 \quad & 2c + 2t +2s \leq 24 \\
& c + 2 t +3s \leq 18\\
& c \geq 0\\
& t \geq 0\\
& s \geq 0
\end{align}
$$

You have to add a third variable to the model, to modify the objective function, and the two constraints. Later, you call the solver again, check the status of the solver, and the solution values.

In [8]:
# Write your own script
from gurobipy import Model, GRB
model = Model()
p = [8,11,15]
c = model.addVar(lb = 0, obj = p[0], vtype = GRB.CONTINUOUS, name = "c")
t = model.addVar(lb = 0, obj = p[1], vtype = GRB.CONTINUOUS, name = "t")
s = model.addVar(lb = 0, obj = p[2], vtype=GRB.CONTINUOUS, name = "s")

model.ModelSense = GRB.MAXIMIZE

A = [[2,2,2],[1,2,3]]
b = [24,18]

# this is good...
# model.addConstr(2*c+2*t +2*s<= 24)
# model.addConstr(c+2*t+3*s <= 18)

# this is better

for Ai, bi in zip(A,b): # zip(A,b) returns list made by [(a1, b1), (a2, b2)...]
  model.addConstr(Ai[0]*c+Ai[1]*t+Ai[2]*s <= bi)


model.update()
model.write("lego_3var.lp")

In [9]:
model.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i3-1005G1 CPU @ 1.20GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 2 rows, 3 columns and 6 nonzeros
Model fingerprint: 0xf32b7170
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [8e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 2e+01]
Presolve time: 0.01s
Presolved: 2 rows, 3 columns, 6 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.4000000e+31   4.500000e+30   3.400000e+01      0s
       2    1.1700000e+02   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.02 seconds (0.00 work units)
Optimal objective  1.170000000e+02


In [10]:
print("status: ", model.Status) # 2 is good
print("objective: ", model.ObjVal)
print("Decision variables:")
print("\t c:", c.X)
print("\t t:", t.X)
print("\t s:", s.X)

status:  2
objective:  117.0
Decision variables:
	 c: 9.0
	 t: 0.0
	 s: 3.0


## 2.3 Random LPs
Let consider the following python function that generates a random LP instance.

Recall that an LP is completely defined by the cost vector $c \in \mathbb{R}^n$, the rhs vector $b \in \mathbb{R}^m$, and the coefficient matrix $A \in \mathbb{R}^{m \times n}$:

$$
    z = \min \,\{ c x \mid Ax \geq b, x \geq 0 \,\}
$$

Look at the following function.

In [11]:
from numpy import random

def RandomLP(n, m, seed=13):
    random.seed(seed)
    c = random.randint(1, 10, size=n)
    b = random.randint(1, 10, size=m)
    A = random.randint(1, 10, size=(m,n))    
    return c, b, A
    
print(RandomLP(3,4))

(array([3, 1, 1]), array([7, 3, 5, 4]), array([[5, 3, 7],
       [6, 5, 3],
       [1, 4, 6],
       [4, 7, 6]]))


Next, we write another function that takes as input the data of an LP instance and builds the LP model using the Gurobi syntax, solves the instances, check the return status of the solver, and, if possible, print to screen an optimal solution.

In [12]:
from gurobipy import Model, GRB, quicksum

def SolveLP(c, b, A):
    m = len(b)
    n = len(c)

    model = Model()

    # Be careful: RangeSet starts at 1 and not a 0, as python list index
    I = range(n)
    J = range(m)

    # we can do this in 42425 different ways, he used lists and zips but it's pretty much the same
    x = []
    for i, ci in enumerate(c):
        x.append(model.addVar(obj=ci, name="x_{}".format(i)))

    for j in J:
        model.addConstr(quicksum(A[j, i]*x[i] for i in I) >= b[j])

    # Default: model.ModelSense = GRB.MINIMIZE

    model.optimize()

    # Basic info about the solution process
    print(f"Status: {model.Status}, ObjVal: {model.ObjVal}")

    xbar = [x[i].X for i in I]
    model.write("general.lp")
    # Return objective value and decision variables
    return model.ObjVal, xbar

Below we show an example of calling this function.

In [13]:
c, b, A = RandomLP(2, 3, 1717)
print(c,b,A)
print(SolveLP(c, b, A))

[2 1] [4 9 8] [[9 2]
 [1 3]
 [9 9]]
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i3-1005G1 CPU @ 1.20GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 3 rows, 2 columns and 6 nonzeros
Model fingerprint: 0x89fb992c
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+00, 9e+00]
Presolve time: 0.01s
Presolved: 3 rows, 2 columns, 6 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.050000e+01   0.000000e+00      0s
       1    3.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.02 seconds (0.00 work units)
Optimal objective  3.000000000e+00
Status: 2, ObjVal: 3.0
(3.0, [0.0, 3.0])


### 2.4 Nurse Scheduling --------------------
Consider the nurse scheduling problem on the lecture' slides.

Let us write the same model, but in addition we use a cost larger than 1 for the Friday, Saturday, and Sunday nights.

For the model description we refer to the slides used during the lecture.

In [ ]:
# Import the numerical python library https://numpy.org/
import numpy as np
from gurobipy import Model, GRB, quicksum

def StaffScheduling(demand, cost, w=5, margin = 1):
    # Data
    d = len(demand)
    # demand = [5, 6, 7, 3, 3, 2, 2]
    # cost = [10.0, 1.0, 1.0, 1.0, 1.5, 5, 1.5]

    # Using the numpy library create a matrix of dimension "d x d"
    A = np.zeros( (d,d) )
    # Set the matrix coefficient to 1 using the patter of the slide
    for i in range(d):
        for j in range(w):
            A[(i+j)%d, i] = 1

    # Declare the model
    
    model = Model()
    # nurses
    x = []
    # money
    # m = model.addVar(obj = 0.2, name = "m", vtype=GRB.CONTINUOUS)
    for i in range(d):
        x.append(model.addVar(lb = 0, obj = 1, name="x_{}".format(i+1), vtype=GRB.INTEGER)) 
    # i look for a min so se don't need to specify anything

    # declare constraints
    for j in range(d): 
        model.addConstr(quicksum(A[j, i]*x[i] for i in range(d)) >= demand[j])
        # model.addConstr( m == cost[j]*quicksum(A[j, i]*x[i] for i in range(d)))
    model.update()
    model.optimize()
    print("nurses optimization: done -----------------------------------------")
    
    o = model.Objval 
    m = model.addVar(obj = 1, name = "m", vtype=GRB.CONTINUOUS)
    
    
    a = 0
    for j in range(d): 
        model.addConstr(quicksum(x) <= o + margin)
        model.addConstr(quicksum(x) >= o - margin)
        a += cost[j]*quicksum(A[j, i]*x[i] for i in range(d))
    model.addConstr( m == a)    
    
    model.update()
    model.optimize()
    
    model.write("nurses.lp")
    status = model.Status
    if status == 2:
        print("cost optimisation: done ---------------------------------------")
        objval = model.Objval
        xbar = [x[i].X for i in range(d)]
        mbar = m.X
        nurses_count = o
        return xbar,status,nurses_count, mbar
    else:
        return status

In [25]:
demand = [5, 6, 7, 3, 3, 2, 2]
cost = [1.0, 1.0, 1.0, 1.0,1.5, 1.5, 1.5]
x, status, nurses_count, mbar = StaffScheduling(demand, cost, margin =3)

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i3-1005G1 CPU @ 1.20GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 7 rows, 7 columns and 35 nonzeros
Model fingerprint: 0xd00cdc7e
Variable types: 0 continuous, 7 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 7e+00]
Found heuristic solution: objective 10.0000000
Presolve time: 0.00s
Presolved: 7 rows, 7 columns, 35 nonzeros
Variable types: 0 continuous, 7 integer (0 binary)

Root relaxation: objective 7.000000e+00, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       7.0000000    7.

In [23]:
days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"] 
for i in range(7):
    print("day:", days[i], "\t Nurses: ", int(x[i]))
    

day: Monday 	 Nurses:  3
day: Tuesday 	 Nurses:  2
day: Wednesday 	 Nurses:  0
day: Thursday 	 Nurses:  0
day: Friday 	 Nurses:  0
day: Saturday 	 Nurses:  0
day: Sunday 	 Nurses:  2


In [19]:
mbar

233.0

### READ FOR EXERCISE 2.5 Exercise: Steel Recycle Blending Problem
The industrial steel can be easily recycled, since it is possible to burn any
scrap to get only liquid steel (without plastics, glasses, ...).
However, it is hard to separate each single metal presents in the scrap,
and as a consequence, beside iron, we can get also chromium, nichel, and
other impurities in the liquid steel.

Depending on the type of production, some metals are desirable, while others
are not. For example, the stainless steel 18/10 must have 18% of chromium and
10% of nichel (consider that chromium and nichel are very expensive, much more
than the steel itself).

**Problem Statement:** Suppose that the Brambilla's Steel company of Voghera can choose to buy some iron
scrap block with different properties regarding the different metals contained in
each block. The company want to produce at minimum cost 100 quintals of stainless
steel 18/10, which must have at least 65% of iron and at most 1% of
impurity materials. Which fraction of each block is going to buy?

The data of the problem are given below.

In [34]:
import numpy as np

# Data of the problem (in theory, read data from .csv or excel file)

# Blocks you can buy
Blocks = ['Block1','Block2','Block3','Block4','Block5','Block6']
n = 6
Weights = [30, 90, 50, 70, 60, 50]  # In quintals
Costs = [50, 100, 80, 85, 92, 115]  # Thousand of euros

# Components of metal in each block (given in percentage)
Cs = np.matrix([[93, 76, 74, 65, 72, 68],  # Ferro
                [5, 13, 11, 16, 6, 23],    # Cromo
                [0, 11, 12, 14, 20, 8],    # Nichel
                [2, 0, 3, 5, 2, 1]])       # Impurità

In [35]:
from gurobipy import Model, GRB, quicksum

Here the decision variables can be chosen in two ways:

a) we can choose $b_i, i=1..6$ as the fraction of the $i$-th block;

b) we can choose $x_i, i = 1..6$ as the weight of the piece of the block chosen (if $x_1 = 10$ we choose $10 q$ out of the $30$ available)

by myself, done choice a). The teacher did b;


The cost can be euro/quintal or euro/block... here we choose the second option

In [65]:
model = Model()
b = []
for i, ci in enumerate(Costs):       
    b.append(model.addVar(lb = 0, ub = 1, obj=ci*Weights[i], name="b_{}".format(i), vtype = GRB.CONTINUOUS))

T = quicksum([quicksum([0.01*Cs[j, i]*Weights[i]*b[i] for i in range(n)]) for j in range(4)])
Ch = quicksum([0.01*Cs[1, i]*Weights[i]*b[i] for i in range(n)])
N = quicksum([0.01*Cs[2, i]*Weights[i]*b[i] for i in range(n)])
Fe = quicksum([0.01*Cs[0, i]*Weights[i]*b[i] for i in range(n)])
Imp = quicksum([0.01*Cs[3, i]*Weights[i]*b[i] for i in range(n)]) 

model.addConstr(T>= 100)
model.addConstr( Ch == 0.18*T ) # CH
model.addConstr( N == 0.10*T ) # NI
model.addConstr(Fe >= 0.65*T ) # FE
model.addConstr( Imp <= 0.01*T ) # IMP
model.update()
model.write("steel.lp")
model.optimize()


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i3-1005G1 CPU @ 1.20GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 5 rows, 6 columns and 28 nonzeros
Model fingerprint: 0x305d14c5
Coefficient statistics:
  Matrix range     [3e-01, 9e+01]
  Objective range  [2e+03, 9e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+02, 1e+02]
Presolve removed 1 rows and 0 columns
Presolve time: 0.01s
Presolved: 4 rows, 6 columns, 23 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.250000e+01   0.000000e+00      0s
       6    1.0567580e+04   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.02 seconds (0.00 work units)
Optimal objective  1.056757991e+04


solved in 5 iterations means that he did 5 iterations of the simplex algorithm (which is basically choosing a vertex of the polytope and going on)

In [63]:
b_ = [b[i].X for i in range(n)]

print("status:", model.Status)
print("objective value", model.ObjVal)
print(b_)

status: 2
objective value 10567.579908675798
[0.0, 0.44647387113140485, 0.0, 0.13698630136986276, 0.030441400304414463, 0.968036529680366]


Version of the teacher: (see github in a couple days)

In [ ]:
model2 = Model()

x = [model2.addVar(ub = Weights[i], obj = c) for i, c in enumerate(Costs)]
model2.addConstr(quicksum(x[i] for i in range(len(x)))==100)
# we assume the cost is by quintals... (in the other is cost by block)
model2.addConstr(quicksum(Cs[0,i]/100*x[i] for i in range(len(x)))>= 65)
model2.addConstr(quicksum(Cs[1,i]/100*x[i] for i in range(len(x)))== 18)
model2.addConstr(quicksum(Cs[2,i]/100*x[i] for i in range(len(x)))== 10)
model2.addConstr(quicksum(Cs[3,i]/100*x[i] for i in range(len(x)))<= 1)

model2.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i3-1005G1 CPU @ 1.20GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 5 rows, 6 columns and 28 nonzeros
Model fingerprint: 0xa5935a04
Coefficient statistics:
  Matrix range     [1e-02, 1e+00]
  Objective range  [5e+01, 1e+02]
  Bounds range     [3e+01, 9e+01]
  RHS range        [1e+00, 1e+02]
Presolve time: 0.01s
Presolved: 5 rows, 6 columns, 28 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.6000000e+03   3.475000e+01   0.000000e+00      0s
       4    1.0567580e+04   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.03 seconds (0.00 work units)
Optimal objective  1.056757991e+04


In [64]:
x_ = [x[i].X for i in range(n)]

print("status:", model2.Status)
print("objective value", model2.ObjVal)
print(x_)

status: 2
objective value 10567.5799086758
[0.0, 39.583333333333336, 0.0, 7.291666666666682, 3.124999999999991, 50.0]


Now consider the case when we can buy only full blocks. What happens? When we use the second model, the thing is that when x (weight) is greater than 0, we need to buy the full block so the price is necessarily full.

We add another binary variable z such that
$$x > 0 \implies z = 1$$
AND $z$ is linked to the price!!!!

We linearize the logical implication as 
$$x \leq Mz$$
where M is a quantity bigger than any other possible value of x (I choose the sum of ALL weights as an upper bound but can be smaller I think, we can also have $x_i \leq M*z_i$ with $M = w_i$

In [57]:
model3= Model()
        # we change obj here, because the cost is not depending on the weights but on the z variable
x = [model3.addVar(ub = Weights[i], obj = 0) for i, c in enumerate(Costs)]
        # we assume cost by weight, so we need to multiply
z = [model3.addVar(vtype = GRB.BINARY, obj = c*Weights[i]) for i, c in enumerate(Costs)]

model3.addConstr(quicksum(x[i] for i in range(len(x)))==100)

model3.addConstr(quicksum(Cs[0,i]/100*x[i] for i in range(len(x)))>= 65)
model3.addConstr(quicksum(Cs[1,i]/100*x[i] for i in range(len(x)))== 18)
model3.addConstr(quicksum(Cs[2,i]/100*x[i] for i in range(len(x)))== 10)
model3.addConstr(quicksum(Cs[3,i]/100*x[i] for i in range(len(x)))<= 1)

# linearised logical impli
M = quicksum(Weights)
for i in range(n):
    model3.addConstr( x[i] <= M*z[i])

model3.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i3-1005G1 CPU @ 1.20GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 11 rows, 12 columns and 40 nonzeros
Model fingerprint: 0xe3596b8b
Variable types: 6 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e-02, 4e+02]
  Objective range  [2e+03, 9e+03]
  Bounds range     [1e+00, 9e+01]
  RHS range        [1e+00, 1e+02]
Presolve removed 3 rows and 2 columns
Presolve time: 0.00s
Presolved: 8 rows, 10 columns, 27 nonzeros
Variable types: 6 continuous, 4 integer (4 binary)
Found heuristic solution: objective 31720.000000
Found heuristic solution: objective 27720.000000

Root relaxation: objective 2.622000e+04, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | 

In [58]:
x_ = [x[i].X for i in range(n)]
z_ = [z[i].X for i in range(n)]

print("status:", model3.Status)
print("objective value", model3.ObjVal)
print("weight vector", x_)
print("price vector", z_)

status: 2
objective value 26220.0
weight vector [0.0, 39.583333333333336, 0.0, 7.291666666666682, 3.124999999999991, 50.0]
price vector [-0.0, 1.0, -0.0, 1.0, 1.0, 1.0]


Last attempt: 

In [74]:
model4 = Model()
b = []
for i, ci in enumerate(Costs):       
    b.append(model4.addVar(obj=ci*Weights[i], name="b_{}".format(i), vtype = GRB.BINARY))

T = quicksum([quicksum([0.01*Cs[j, i]*Weights[i]*b[i] for i in range(n)]) for j in range(4)])
Ch = quicksum([0.01*Cs[1, i]*Weights[i]*b[i] for i in range(n)])
N = quicksum([0.01*Cs[2, i]*Weights[i]*b[i] for i in range(n)])
Fe = quicksum([0.01*Cs[0, i]*Weights[i]*b[i] for i in range(n)])
Imp = quicksum([0.01*Cs[3, i]*Weights[i]*b[i] for i in range(n)]) 

model4.addConstr(T >= 100)
model4.addConstr( Ch == 0.18*T ) # CH
model4.addConstr( N == 0.10*T ) # NI
model4.addConstr(Fe >= 0.65*T ) # FE
model4.addConstr( Imp <= 0.01*T ) # IMP
model4.update()
# model4.write("steel.lp")
model4.optimize()


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i3-1005G1 CPU @ 1.20GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 5 rows, 6 columns and 28 nonzeros
Model fingerprint: 0x1e392dab
Variable types: 0 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [3e-01, 9e+01]
  Objective range  [2e+03, 9e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+02, 1e+02]
Presolve removed 1 rows and 6 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 4 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -


We see that the objective value increases (makes sense, since we are )

**EXERCISE 3:** First, write on paper a LP model to solve the steel production problem for the Rossi's Steel company

**EXERCISE 4:** Solve the LP using Gurobi. 

### Exercise 2.6: The Magic Square Puzzle
The [Magic Square](https://en.wikipedia.org/wiki/Magic_square) puzzle asks to place into a grid of size $n \times n$ the digits from $1$ to $n^2$, in such a way that the sum of the digits in each row, the sum of digits in each column, and the sum of the digits on the two main diagonals, is equal to the same number.

You can play with a $4 \times 4$ puzzle on Google Sheet online at [magic square link](https://docs.google.com/spreadsheets/d/1OcicQdKbZXpSV4ooXsbGC2OFR5cSgwgWgUimdwcT0qA/edit?usp=sharing).

**EXERCISE 5:** Write an ILP model to solve the magic square puzzle of size $n$.

In [18]:
# Write your model here